In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


##### Please run this code with the kernel reinvent.v3.2

In [10]:
# load dependencies
import sys
import pickle
import os
import shutil
import json
import pandas as pd
import numpy as np
import rdkit.Chem as Chem
from numpy.random import default_rng
import torch
from ast import literal_eval
from torch import nn, optim
from rdkit import Chem
from rdkit.Chem import AllChem
from tdc import Oracle
import subprocess

### First, we need to install the custom reinvent scoring package to support the Bradley-Terry model

In [11]:
# print python version
import sys

# Print Python version
print(f"Python version: {sys.version}")

Python version: 3.7.6 | packaged by conda-forge | (default, Jun  1 2020, 18:57:50) 
[GCC 7.5.0]


In [12]:
! pip show reinvent_scoring

Name: reinvent-scoring
Version: 0.0.73
Summary: Scoring functions for Reinvent
Home-page: https://github.com/MolecularAI/reinvent-scoring.git
Author: MolecularAI
Author-email: patronov@gmail.com
License: UNKNOWN
Location: /home/springnuance/reinvent-hitl/reinvent-scoring
Requires: 
Required-by: 


In [13]:
# ! conda install scikit-learn=0.21.3

##### If there already exists reinvent_scoring, we should uninstall it

In [14]:
! pip uninstall -y reinvent_scoring

Found existing installation: reinvent-scoring 0.0.73
Uninstalling reinvent-scoring-0.0.73:
  Successfully uninstalled reinvent-scoring-0.0.73


##### Now we install the custom reinvent scoring package
##### The flag -e means that the package is installed in editable mode, so that changes to the code will be immediately available without reinstalling the package. All package info is stored in the setup.py file.

In [15]:
! pip install -e "/home/springnuance/reinvent-hitl/reinvent-scoring"
! pip install -e "/home/springnuance/reinvent-hitl/reinvent-chemistry"
! pip install -e "/home/springnuance/reinvent-hitl/reinvent-models"

Obtaining file:///home/springnuance/reinvent-hitl/reinvent-scoring
  Preparing metadata (setup.py) ... done
  Running setup.py develop for reinvent-scoring
Obtaining file:///home/springnuance/reinvent-hitl/reinvent-chemistry
  Preparing metadata (setup.py) ... done
  Attempting uninstall: reinvent-chemistry
    Found existing installation: reinvent-chemistry 0.0.51
    Uninstalling reinvent-chemistry-0.0.51:
      Successfully uninstalled reinvent-chemistry-0.0.51
  Running setup.py develop for reinvent-chemistry
Obtaining file:///home/springnuance/reinvent-hitl/reinvent-models
  Preparing metadata (setup.py) ... done
  Attempting uninstall: reinvent-models
    Found existing installation: reinvent-models 0.0.15rc1
    Uninstalling reinvent-models-0.0.15rc1:
      Successfully uninstalled reinvent-models-0.0.15rc1
  Running setup.py develop for reinvent-models


In [16]:
# ! conda install -y scikit-learn=0.21.3
! pip list | grep reinvent_scoring

In [17]:
from training_Bradley_Terry_model.bradley_terry import BradleyTerryModel
from training_Rank_ListNet_model.rank_listnet import RankListNetModel
from training_Score_Regression_model.score_regression import ScoreRegressionModel
from helper import load_drd2_dataset, write_REINVENT_config, change_config_json, \
                    read_scaffold_result, load_feedback_model, smiles_human_score, \
                    compute_fingerprints, retrain_feedback_model,\
                    create_drd2_dataset, combine_drd2_dataset, save_drd2_dataset
                        
from scripts.acquisition import select_query_feedback
from scripts.acquisition import predict_proba_from_model

def check_create(path):
    """
    Check if the directory exists, if not, create it.
    """
    if not os.path.exists(path):
        os.makedirs(path)
        
def run_HITL_classify(
        seed, reinvent_dir, reinvent_env, output_dir, 
        feedback_type, # scoring, comparing, ranking
        base_training_dataset_path, # path to the base training dataset
        model_pretrained_path, # Path to the pretrained model before REINVENT_round_1
        model_pretrained_name, # Name of the pretrained model before REINVENT_round_1
        num_rounds, # number of rounds, corresponding to R in the paper
        num_iters, # number of iterations of showing molecules to the human for feedback at each round, corresponding to T in the paper
        num_queries, # number of molecules shown to the simulated chemist at each iteration
        REINVENT_n_steps, # number of REINVENT optimization steps. This is not related to the HITL but on the REINVENT side
        batch_size, # batch size of the reinforcement learning model, or size of scaffold_memory.csv
        acquisition, # acquisition: 'uncertainty', 'random', 'thompson', 'greedy' 
        sigma_noise, # noise level for simulated chemist's responses
        choose_top_smiles, # number of top scoring molecules to choose for feedback
        training_epochs, # number of epochs for training the model in each HITL iteration
        learning_rate, # learning rate for training the model in each HITL iteration
        ):

    np.random.seed(seed)
    rng = default_rng(seed)
    
    ################################################
    # DEFINING REINVENT JOBNAME, JOBID, OUTPUT_DIR #
    ################################################

    jobname = "fine-tune predictive component HITL"
    jobid = output_dir
    conf_filename = "config.json"

    # create root output dir
    check_create(output_dir)

    # create HITL round folders to store results
    for REINVENT_round in range(1, num_rounds + 1):
        check_create(f"{output_dir}/REINVENT_round_{REINVENT_round}")
        for HITL_iteration in range(1, num_iters + 1):
            check_create(f"{output_dir}/REINVENT_round_{REINVENT_round}/HITL_iteration_{HITL_iteration}")
    
    # multi-parameter optimization (MPO) loop
    print(f"\nRunning DRD2 (one objective) with rounds {num_rounds}, iters {num_iters}, queries {num_queries}, seed {seed}")
    print(f"Results will be saved at {output_dir}")
    
    base_training_dataset_outputs = load_drd2_dataset(feedback_type=feedback_type, 
                                                 data_path=base_training_dataset_path)

    
    print("Loading initial training datasets successfully")

    # ########################### REINVENT rounds ######################################

    for REINVENT_round in range(1, num_rounds + 1):

        print("=====================================")
        print(f"REINVENT round = {REINVENT_round}")
        
        configuration_JSON_path = write_REINVENT_config(feedback_type, reinvent_dir, jobid, jobname, 
                                REINVENT_round_output_dir=f"{output_dir}/REINVENT_round_{REINVENT_round}", 
                                conf_filename=conf_filename)

        print(f"Creating config file: {configuration_JSON_path}.")

        configuration = json.load(open(f"{configuration_JSON_path}"))

        if REINVENT_round == 1:
            current_model_path = model_pretrained_path
        else:
            # The last iteration's model from the previous round
            current_model_path = f"{output_dir}/REINVENT_round_{REINVENT_round - 1}/HITL_iteration_{num_iters}/{model_pretrained_name}"

        configuration = change_config_json(configuration, REINVENT_n_steps, batch_size, current_model_path)

        # write the updated configuration file 

        with open(configuration_JSON_path, 'w') as f:
            json.dump(configuration, f, indent=4, sort_keys=True)
    
        print("Run REINVENT")                
        command = f"{reinvent_env}/bin/python"
        script = f"{reinvent_dir}/input.py"
        stderr_file = f"{output_dir}/REINVENT_round_{REINVENT_round}/run.err"
        stdout_file = f"{output_dir}/REINVENT_round_{REINVENT_round}/run.out"

        # Construct the full command to run
        cmd = [command, script, configuration_JSON_path]
        # Open the file to which you want to redirect stderr and stdout
        with open(stderr_file, 'w') as ferr, open(stdout_file, 'w') as fout:
            # Execute the command
            result = subprocess.run(cmd, text=True, stdout=fout, stderr=ferr)
        # Check the result
        print("Exit code:", result.returncode)
        
        #############################################################################
        # REINVENT HAS OUTPUT THE RESULT in path f"{output_dir}/REINVENT_round_{REINVENT_round}/results" #
        #############################################################################
        
        # Get the high scoring molecules
        output_high_score = read_scaffold_result(f"{output_dir}/REINVENT_round_{REINVENT_round}/results/scaffold_memory.csv", 
                                                 choose_top_smiles=choose_top_smiles)
        
        scaffold_df = output_high_score["scaffold_df"]
        smiles = output_high_score["smiles"]
        
        # store molecule indexes selected for feedback
        selected_feedback = np.empty(0).astype(int)

        ########################### HITL_iteration in each REINVENT round #####################
        
        for HITL_iteration in range(1, num_iters + 1): # T number of HITL_iterations

            print("----------------------------------")
            print(f"HITL iteration = {HITL_iteration}")
            
            # Loading feedback model
            if REINVENT_round == 1 and HITL_iteration == 1:
                feedback_model_path = model_pretrained_path
            elif REINVENT_round != 1 and HITL_iteration == 1:
                # The last iteration's model from the previous round
                feedback_model_path = f"{output_dir}/REINVENT_round_{REINVENT_round - 1}/HITL_iteration_{num_iters}/{model_pretrained_name}"
            else:
                # The previous HITL iteration's model from current round
                feedback_model_path = f"{output_dir}/REINVENT_round_{REINVENT_round}/HITL_iteration_{HITL_iteration - 1}/{model_pretrained_name}"
            
            feedback_model = load_feedback_model(feedback_type, feedback_model_path)
            
            #print(f"The predicted scores are")
            #predicted_scores = predict_proba_from_model(feedback_type, feedback_model, smiles)
            #print(predicted_scores)

            ######################################################## 
            # Select queries number of smiles with Active Learning #
            ########################################################

            if len(smiles) > num_queries:
                new_queried_smiles_indices = select_query_feedback(feedback_type, feedback_model, 
                                                  scaffold_df, num_queries, list(smiles), 
                                                  selected_feedback, acquisition, rng) 
            else:
                new_queried_smiles_indices = select_query_feedback(feedback_type, feedback_model, 
                                                  scaffold_df, len(smiles), list(smiles), 
                                                  selected_feedback, acquisition, rng)
            
            #print(f"Feedback idx at HITL iteration {HITL_iteration}: {new_queried_smiles_indices}")
            
            new_queried_smiles = [smiles[i] for i in new_queried_smiles_indices]
  
            selected_feedback = np.hstack((selected_feedback, new_queried_smiles_indices))

            new_queried_smiles_human_score = smiles_human_score(new_queried_smiles, sigma_noise)
            
            print(f"Human score at HITL iteration {HITL_iteration}: {new_queried_smiles_human_score}")
            
            # use the augmented training data to retrain the model
            new_queried_fps = np.array([compute_fingerprints(smiles) for smiles in new_queried_smiles])

            iteration_training_dataset_outputs = create_drd2_dataset(feedback_type, 
                                                                      new_queried_smiles, 
                                                                      new_queried_smiles_human_score,
                                                                      new_queried_fps)
            
            if feedback_type == "scoring":
                print(f"New queried dataset size: {len(iteration_training_dataset_outputs['smiles'])}")
            else:
                print(f"New queried dataset size: {len(iteration_training_dataset_outputs['smiles_1'])}")
            
            # combining the base training dataset with the new queried dataset
            base_training_dataset_outputs = combine_drd2_dataset(feedback_type, base_training_dataset_outputs, 
                                                                   iteration_training_dataset_outputs)
            
            if feedback_type == "scoring":
                print(f"Combined dataset size: {len(base_training_dataset_outputs['smiles'])}")
            else:
                print(f"Combined dataset size: {len(base_training_dataset_outputs['smiles_1'])}")
            
            # save augmented training data
            save_drd2_dataset(feedback_type, iteration_training_dataset_outputs, f"{output_dir}/REINVENT_round_{REINVENT_round}/HITL_iteration_{HITL_iteration}/iteration_queried_data.csv")
            save_drd2_dataset(feedback_type, base_training_dataset_outputs, f"{output_dir}/REINVENT_round_{REINVENT_round}/HITL_iteration_{HITL_iteration}/iteration_combined_data.csv")
            
            print(f"Saved augmented training data at {output_dir}/REINVENT_round_{REINVENT_round}/HITL_iteration_{HITL_iteration}/iteration_combined_data.csv")
            
            # Retraining the feedback model using the augmented train set
            retrained_feedback_model = retrain_feedback_model(feedback_type, feedback_model, 
                                                              base_training_dataset_outputs,
                                                              training_epochs, learning_rate)
            
            # save the retrained feedback model
            feedback_model_saving_path = f"{output_dir}/REINVENT_round_{REINVENT_round}/HITL_iteration_{HITL_iteration}/{model_pretrained_name}"
            torch.save(retrained_feedback_model.state_dict(), feedback_model_saving_path)
  
            print(f"Saved retrained feedback model at {feedback_model_saving_path}")

In [18]:
print(os.getcwd())

/home/springnuance/reinvent-hitl/Base-Code-Binh


### Running score regression model

In [19]:
! pip install -e "/home/springnuance/reinvent-hitl/reinvent-scoring"

from training_Bradley_Terry_model.bradley_terry import BradleyTerryModel
from training_Rank_ListNet_model.rank_listnet import RankListNetModel
from training_Score_Regression_model.score_regression import ScoreRegressionModel
from helper import load_drd2_dataset, write_REINVENT_config, change_config_json, \
                    read_scaffold_result, load_feedback_model, smiles_human_score, \
                    compute_fingerprints, retrain_feedback_model,\
                    create_drd2_dataset, combine_drd2_dataset, save_drd2_dataset
                        
from scripts.acquisition import select_query_feedback
from scripts.acquisition import predict_proba_from_model

seed = 42

# change these path variables as required
reinvent_dir = "/home/springnuance/reinvent-hitl/Reinvent" # We must use absolute path
reinvent_env = "/home/springnuance/miniconda3/envs/ReinventCommunity" # We must use absolute path

# the performance of the initial model should not be good. Specifically, it should work at 0.5 accuracy 
# If the model is too good, retrain the model to become weaker, we are trying to make the model to learn via HITL

feedback_type = "scoring" # scoring, comparing, ranking

# feedback type as scoring:
# Given a molecule, what is the probability that the molecule is active regarding DRD2?  

base_training_dataset_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Score_Regression_model/small_drd2_training_data.csv"

model_pretrained_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Score_Regression_model/score_regression_model.pth"
model_pretrained_name = "score_regression_model.pth"

num_rounds = 3 # number of rounds, corresponding to R in the paper
num_iters = 5 # number of iterations of showing molecules to the human for feedback at each round
REINVENT_n_steps = 100 # number of REINVENT optimization steps
batch_size = 128 # batch size of the reinforcement learning model, or size of scaffold_memory.csv

num_queries = 20 # number of molecules, pairs or a set of molecules, dependig on the task, 
                 # shown to the simulated chemist at each HITL_iteration

choose_top_smiles = 150 # number of top molecules to choose from scaffold. 

training_epochs = 10 # number of epochs for training the model in each HITL iteration
learning_rate = 0.000001 # learning rate for training the model in each HITL iteration

# Case 1
acquisition = "random"
sigma_noise = 0.0

# Case 2
# acquisition = "uncertainty"
# sigma_noise = 0.0

# Case 3
# acquisition = "greedy"
# sigma_noise = 0.0

# Case 4
# acquisition = "random"
# sigma_noise = 0.1

# Case 5
# acquisition = "uncertainty"
# sigma_noise = 0.1

# Case 6
# acquisition = "greedy"
# sigma_noise = 0.1

output_dir = f"output_score_regression/R{num_rounds}_T{num_iters}_Q{num_queries}_acq_{acquisition}_noise_{sigma_noise}"

run_HITL_classify(
        seed, reinvent_dir, reinvent_env, output_dir,
        feedback_type, # scoring, comparing, ranking
        base_training_dataset_path, # Path to the pretrained model before REINVENT_round_1/HITL_iteration_1
        model_pretrained_path, # Path to the pretrained model before REINVENT_round_1
        model_pretrained_name, # Name of the pretrained model before REINVENT_round_1
        num_rounds, # number of rounds, corresponding to R in the paper
        num_iters, # number of iterations of showing molecules to the human for feedback at each round, corresponding to T in the paper
        num_queries, # number of molecules shown to the simulated chemist at each HITL_iteration
        REINVENT_n_steps, # number of REINVENT optimization steps
        batch_size, # batch size of the reinforcement learning model, or size of scaffold_memory.csv
        acquisition, # acquisition: 'uncertainty', 'random', 'thompson', 'greedy' (if None run with no human interaction)
        sigma_noise, # noise level for simulated chemist's responses
        choose_top_smiles, # number of top scoring molecules to choose for feedback
        training_epochs, # number of epochs for training the model in each HITL iteration
        learning_rate, # learning rate for training the model in each HITL iteration
)


Obtaining file:///home/springnuance/reinvent-hitl/reinvent-scoring
  Preparing metadata (setup.py) ... done
  Attempting uninstall: reinvent-scoring
    Found existing installation: reinvent-scoring 0.0.73
    Uninstalling reinvent-scoring-0.0.73:
      Successfully uninstalled reinvent-scoring-0.0.73
  Running setup.py develop for reinvent-scoring

Running DRD2 (one objective) with rounds 3, iters 5, queries 20, seed 42
Results will be saved at output_score_regression/R3_T5_Q20_acq_random_noise_0.0
Loading initial training datasets successfully
REINVENT round = 1
Creating config file: output_score_regression/R3_T5_Q20_acq_random_noise_0.0/REINVENT_round_1/config.json.
Run REINVENT


### Running Bradley Terry model

In [18]:
! pip install -e "/home/springnuance/reinvent-hitl/reinvent-scoring"

seed = 42
restart = False # If restart is True, we would rerun everything
                # If restart is False, we would continue from the latest found HITL_iteration
                    
# change these path variables as required
reinvent_dir = "/home/springnuance/reinvent-hitl/Reinvent" # We must use absolute path
reinvent_env = "/home/springnuance/miniconda3/envs/ReinventCommunity" # We must use absolute path

# the performance of the initial model should not be good. Specifically, it should work at 0.5 accuracy 
# If the model is too good, retrain the model to become weaker, we are trying to make the model to learn via HITL

feedback_type = "comparing" # scoring, comparing, ranking

# feedback type as comparing:
# Given two molecules, what is the probability that the first molecule is more active than the second molecule regarding DRD2?

base_training_dataset_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Bradley_Terry_model/small_drd2_training_data.csv"

model_pretrained_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Bradley_Terry_model/bradley_terry_model.pth"
model_pretrained_name = "bradley_terry_model.pth"

num_rounds = 3 # number of rounds, corresponding to R in the paper
num_iters = 3 # number of iterations of showing molecules to the human for feedback at each round
REINVENT_n_steps = 50 # number of REINVENT optimization steps
batch_size = 64 # batch size of the reinforcement learning model, or size of scaffold_memory.csv

num_queries = 5 # number of molecules, pairs or a set of molecules, dependig on the task, 
                 # shown to the simulated chemist at each HITL_iteration

# The actual queries is actually (num_queries) x (num_queries - 1), since we are comparing each pair of molecules
# As a result, 5 x 4 = 20 pairs of molecules are shown to the simulated chemist at each HITL_iteration

choose_top_smiles = 200 # number of top molecules to choose from scaffold. 

training_epochs = 10 # number of epochs for training the model in each HITL iteration
learning_rate = 0.00001 # learning rate for training the model in each HITL iteration

# Case 1
acquisition = "random"
sigma_noise = 0.0

# Case 2
# acquisition = "uncertainty"
# sigma_noise = 0.0

# Case 3
# acquisition = "greedy"
# sigma_noise = 0.0

# Case 4
# acquisition = "random"
# sigma_noise = 0.1

# Case 5
# acquisition = "uncertainty"
# sigma_noise = 0.1

# Case 6
# acquisition = "greedy"
# sigma_noise = 0.1

output_dir = f"output_bradley_terry/R{num_rounds}_T{num_iters}_Q{num_queries}_acq_{acquisition}_noise_{sigma_noise}"

run_HITL_classify(
        seed, reinvent_dir, reinvent_env, output_dir,
        feedback_type, # scoring, comparing, ranking
        base_training_dataset_path, # Path to the pretrained model before REINVENT_round_1/HITL_iteration_1
        model_pretrained_path, # Path to the pretrained model before REINVENT_round_1
        model_pretrained_name, # Name of the pretrained model before REINVENT_round_1
        num_rounds, # number of rounds, corresponding to R in the paper
        num_iters, # number of iterations of showing molecules to the human for feedback at each round, corresponding to T in the paper
        num_queries, # number of molecules shown to the simulated chemist at each HITL_iteration
        REINVENT_n_steps, # number of REINVENT optimization steps
        batch_size, # batch size of the reinforcement learning model, or size of scaffold_memory.csv
        acquisition, # acquisition: 'uncertainty', 'random', 'thompson', 'greedy' (if None run with no human interaction)
        sigma_noise, # noise level for simulated chemist's responses
        choose_top_smiles, # number of top scoring molecules to choose for feedback
        training_epochs, # number of epochs for training the model in each HITL iteration
        learning_rate, # learning rate for training the model in each HITL iteration
)

Obtaining file:///home/springnuance/reinvent-hitl/reinvent-scoring
  Preparing metadata (setup.py) ... done
  Attempting uninstall: reinvent-scoring
    Found existing installation: reinvent-scoring 0.0.73
    Uninstalling reinvent-scoring-0.0.73:
      Successfully uninstalled reinvent-scoring-0.0.73
  Running setup.py develop for reinvent-scoring

Running DRD2 (one objective) with rounds 2, iters 3, queries 10, seed 42
Results will be saved at output_bradley_terry_R2_T3_Q10_random_s0.1
Loading initial training and testing datasets successfully
REINVENT round = 1
Creating config file: output_bradley_terry_R2_T3_Q10_random_s0.1/REINVENT_round_1/config.json.
Run REINVENT
Exit code: 0
Number of SMILES in scaffold_memory.csv:  125
----------------------------------
HITL iteration = 1
Loading Bradley Terry model successfully from output_bradley_terry_R2_T3_Q10_random_s0.1/REINVENT_round_1/HITL_iteration_1/bradley_terry_model.pth
The predicted scores are


Found local copy...


[0.76 0.78 0.88 0.34 0.37 0.42 0.77 0.84 0.02 0.82 0.6  0.46 0.48 0.27
 0.96 0.93 0.29 0.01 0.11 0.75 0.15 0.57 0.18 0.3  0.91 0.68 0.09 0.99
 0.55 0.26 0.17 0.54 0.63 0.07 0.62 0.19 0.73 0.49 0.33 0.98 0.8  0.44
 0.22 0.53 0.24 0.89 0.56 0.58 0.21 0.9  0.14 0.03 0.81 0.05 0.69 0.36
 0.85 0.06 0.41 0.2  0.86 0.1  0.52 0.16 0.32 0.38 0.35 0.95 0.5  0.97
 0.71 0.25 0.79 0.   0.28 0.31 0.12 0.59 0.51 0.67 0.65 0.4  0.13 0.47
 0.94 0.23 0.7  0.87 0.45 0.83 0.61 0.08 0.04 0.74 0.43 0.39 0.92 0.66
 0.72 0.64]
Feedback idx at HITL iteration 1: [96 71  8 60 41 94 68  9 19 82]
Human score at HITL iteration 1: [0.1269721185961924, 0.013241194322854566, 0.07527932731986266, 0.15800174675186307, 0.0, 0.0, 0.17240752781177662, 0.07774969003008159, 0.0, 0.07406618016218973]
New queried dataset size: 90
Combined dataset size: 114
Saved augmented training data at output_bradley_terry_R2_T3_Q10_random_s0.1/REINVENT_round_1/HITL_iteration_1/iteration_combined_data.csv
Epoch 1, Loss: 0.027705786228179933

Found local copy...


[0.54 0.58 0.83 0.57 0.24 0.48 0.77 0.82 0.05 0.94 0.59 0.52 0.29 0.23
 0.96 0.93 0.32 0.   0.03 0.78 0.16 0.46 0.18 0.14 0.71 0.75 0.04 0.99
 0.5  0.12 0.21 0.56 0.81 0.06 0.37 0.41 0.84 0.17 0.34 0.97 0.51 0.43
 0.2  0.31 0.28 0.89 0.26 0.68 0.36 0.86 0.07 0.01 0.64 0.09 0.42 0.55
 0.66 0.08 0.73 0.33 0.85 0.4  0.49 0.1  0.39 0.45 0.44 0.95 0.76 0.98
 0.65 0.38 0.91 0.02 0.22 0.3  0.19 0.79 0.74 0.61 0.7  0.69 0.13 0.53
 0.9  0.47 0.62 0.8  0.27 0.67 0.63 0.15 0.11 0.87 0.25 0.35 0.88 0.6
 0.92 0.72]
Feedback idx at HITL iteration 2: [74 38 33 61 80 17 39 45 53 86]
Human score at HITL iteration 2: [0.0, 0.6946837357173267, 0.026914087167113385, 0.0, 0.0, 0.0, 0.0, 0.0347931033579309, 0.0, 0.0]
New queried dataset size: 90
Combined dataset size: 204
Saved augmented training data at output_bradley_terry_R2_T3_Q10_random_s0.1/REINVENT_round_1/HITL_iteration_2/iteration_combined_data.csv
Epoch 1, Loss: 0.22836504876613617
Epoch 2, Loss: 0.22707618276278177
Epoch 3, Loss: 0.22553670406341

...

[0.89 0.67 0.88 0.   0.73 0.87 0.84 0.41 0.51 0.92 0.96 0.08 0.54 0.48
 0.52 0.21 0.47 0.1  0.42 0.94 0.37 0.46 0.95 0.98 0.43 0.24 0.19 0.03
 0.06 0.72 0.69 0.4  0.65 0.15 0.79 0.82 0.66 0.18 0.68 0.44 0.8  0.2
 0.78 0.93 0.31 0.76 0.6  0.59 0.56 0.45 0.63 0.28 0.38 0.99 0.33 0.57
 0.62 0.36 0.16 0.17 0.83 0.26 0.64 0.14 0.34 0.07 0.61 0.09 0.97 0.85
 0.13 0.04 0.02 0.32 0.91 0.74 0.39 0.29 0.53 0.05 0.77 0.3  0.23 0.25
 0.49 0.7  0.58 0.22 0.27 0.81 0.5  0.9  0.55 0.11 0.71 0.35 0.86 0.12
 0.01 0.75]
Feedback idx at HITL iteration 1: [93 40 70 32 63 68 35 90 14 86]
Human score at HITL iteration 1: [0.0, 0.1857486870860026, 0.0, 0.0, 0.14476864177760487, 0.0, 0.029030004313523563, 0.0, 0.0, 0.02714068769328097]
New queried dataset size: 90
Combined dataset size: 384
Saved augmented training data at output_bradley_terry_R2_T3_Q10_random_s0.1/REINVENT_round_2/HITL_iteration_1/iteration_combined_data.csv
Epoch 1, Loss: 0.059016840532422066
Epoch 2, Loss: 0.05746757425367832
Epoch 3, Loss

Found local copy...


[0.86 0.64 0.81 0.   0.5  0.88 0.83 0.33 0.55 0.85 0.96 0.04 0.26 0.21
 0.49 0.06 0.56 0.4  0.43 0.91 0.44 0.32 0.93 0.99 0.51 0.16 0.19 0.11
 0.23 0.82 0.84 0.52 0.48 0.3  0.72 0.79 0.57 0.1  0.71 0.39 0.68 0.27
 0.75 0.95 0.58 0.67 0.47 0.74 0.37 0.66 0.69 0.18 0.38 0.98 0.22 0.9
 0.54 0.17 0.12 0.09 0.94 0.2  0.77 0.35 0.34 0.08 0.65 0.28 0.97 0.73
 0.29 0.01 0.03 0.7  0.87 0.53 0.31 0.42 0.59 0.05 0.63 0.36 0.25 0.13
 0.41 0.62 0.45 0.15 0.46 0.78 0.76 0.92 0.61 0.07 0.6  0.14 0.89 0.24
 0.02 0.8 ]
Feedback idx at HITL iteration 2: [69 80 19 60 87 98 41 13 50 27]
Human score at HITL iteration 2: [0.08708186312199102, 0.018000169850158978, 0.0, 0.012214618096354636, 0.0, 0.0, 0.0, 0.11128798652543538, 0.052200784324358096, 0.0]
New queried dataset size: 90
Combined dataset size: 474
Saved augmented training data at output_bradley_terry_R2_T3_Q10_random_s0.1/REINVENT_round_2/HITL_iteration_2/iteration_combined_data.csv
Epoch 1, Loss: 0.17544675102600685
Epoch 2, Loss: 0.1688397641365

Found local copy...


[0.83 0.66 0.68 0.01 0.37 0.88 0.87 0.36 0.6  0.89 0.94 0.05 0.19 0.17
 0.49 0.13 0.59 0.62 0.46 0.85 0.44 0.51 0.8  0.98 0.61 0.23 0.2  0.06
 0.56 0.86 0.93 0.55 0.21 0.32 0.54 0.77 0.52 0.09 0.75 0.25 0.58 0.28
 0.73 0.91 0.78 0.47 0.18 0.72 0.07 0.48 0.63 0.22 0.35 0.99 0.31 0.95
 0.34 0.24 0.11 0.15 0.96 0.12 0.82 0.45 0.39 0.1  0.57 0.67 0.97 0.7
 0.38 0.02 0.   0.92 0.76 0.41 0.27 0.33 0.71 0.04 0.42 0.29 0.3  0.08
 0.43 0.65 0.5  0.4  0.53 0.81 0.79 0.84 0.64 0.14 0.69 0.16 0.9  0.26
 0.03 0.74]
Feedback idx at HITL iteration 3: [58 23  0 17 20 10 56 77 99 76]
Human score at HITL iteration 3: [0.03256040307755542, 0.0, 0.0, 0.061594290413728006, 0.1039555602997674, 0.09901596277656086, 0.0, 0.0, 0.03623535247633585, 0.09937156611033844]
New queried dataset size: 90
Combined dataset size: 564
Saved augmented training data at output_bradley_terry_R2_T3_Q10_random_s0.1/REINVENT_round_2/HITL_iteration_3/iteration_combined_data.csv
Epoch 1, Loss: 0.09148519543501046
Epoch 2, Loss: 0.

### Running Rank ListNet model

In [19]:
! pip install -e "/home/springnuance/reinvent-hitl/reinvent-scoring"

seed = 42
restart = False # If restart is True, we would rerun everything
                # If restart is False, we would continue from the latest found HITL_iteration
                
# change these path variables as required
reinvent_dir = os.path.expanduser("/home/springnuance/reinvent-hitl/Reinvent") # We must use absolute path
reinvent_env = os.path.expanduser("/home/springnuance/miniconda3/envs/ReinventCommunity") # We must use absolute path

# the performance of the initial model should not be good. Specifically, it should work at 0.5 accuracy 
# If the model is too good, retrain the model to become weaker, we are trying to make the model to learn via HITL

feedback_type = "ranking" # scoring, comparing, ranking

# feedback type as ranking:
# Given N molecules, what are the orders of preference of these molecules regarding DRD2?

base_training_dataset_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Rank_ListNet_model/small_drd2_training_data.csv"

model_pretrained_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Rank_ListNet_model/rank_listnet_model.pth"
model_pretrained_name = "rank_listnet_model.pth"

num_rounds = 3 # number of rounds, corresponding to R in the paper
num_iters = 5 # number of iterations of showing molecules to the human for feedback at each round
REINVENT_n_steps = 50 # number of REINVENT optimization steps
batch_size = 64 # batch size of the reinforcement learning model, or size of scaffold_memory.csv

num_queries = 6 # number of molecules, pairs or a set of molecules, dependig on the task, 
                 # shown to the simulated chemist at each HITL_iteration

# The actual queries is actually binom(num_queries, 3), since we are ranking each set of 3 molecules
# As a result, binom(6, 3) = 20 sets of molecules are shown to the simulated chemist at each HITL_iteration

choose_top_smiles = 200 # number of top molecules to choose from scaffold. 

training_epochs = 10 # number of epochs for training the model in each HITL iteration
learning_rate = 0.00001 # learning rate for training the model in each HITL iteration

# Case 1
acquisition = "random"
sigma_noise = 0.0

# Case 2
# acquisition = "uncertainty"
# sigma_noise = 0.0

# Case 3
# acquisition = "greedy"
# sigma_noise = 0.0

# Case 4
# acquisition = "random"
# sigma_noise = 0.1

# Case 5
# acquisition = "uncertainty"
# sigma_noise = 0.1

# Case 6
# acquisition = "greedy"
# sigma_noise = 0.1

output_dir = f"output_rank_listnet/R{num_rounds}_T{num_iters}_Q{num_queries}_acq_{acquisition}_noise_{sigma_noise}"

run_HITL_classify(
        seed, reinvent_dir, reinvent_env, output_dir,
        feedback_type, # scoring, comparing, ranking
        base_training_dataset_path, # Path to the pretrained model before REINVENT_round_1/HITL_iteration_1
        model_pretrained_path, # Path to the pretrained model before REINVENT_round_1
        model_pretrained_name, # Name of the pretrained model before REINVENT_round_1
        num_rounds, # number of rounds, corresponding to R in the paper
        num_iters, # number of iterations of showing molecules to the human for feedback at each round, corresponding to T in the paper
        num_queries, # number of molecules shown to the simulated chemist at each HITL_iteration
        REINVENT_n_steps, # number of REINVENT optimization steps
        batch_size, # batch size of the reinforcement learning model, or size of scaffold_memory.csv
        acquisition, # acquisition: 'uncertainty', 'random', 'thompson', 'greedy' (if None run with no human interaction)
        sigma_noise, # noise level for simulated chemist's responses
        choose_top_smiles, # number of top scoring molecules to choose for feedback
        training_epochs, # number of epochs for training the model in each HITL iteration
        learning_rate, # learning rate for training the model in each HITL iteration
)

Obtaining file:///home/springnuance/reinvent-hitl/reinvent-scoring
  Preparing metadata (setup.py) ... done
  Attempting uninstall: reinvent-scoring
    Found existing installation: reinvent-scoring 0.0.73
    Uninstalling reinvent-scoring-0.0.73:
      Successfully uninstalled reinvent-scoring-0.0.73
  Running setup.py develop for reinvent-scoring

Running DRD2 (one objective) with rounds 10, iters 5, queries 20, seed 42
Results will be saved at output_rank_listnet_R10_T5_Q20_random_s0.0
Loading initial training and testing datasets successfully
REINVENT round = 1
Creating config file: output_rank_listnet_R10_T5_Q20_random_s0.0/REINVENT_round_1/config.json.
Run REINVENT
Exit code: 0
Number of SMILES in scaffold_memory.csv:  786
----------------------------------
HITL iteration = 1
Loading Rank ListNet model successfully from output_rank_listnet_R10_T5_Q20_random_s0.0/REINVENT_round_1/HITL_iteration_1/rank_listnet_model.pth
The predicted scores are


Found local copy...


[0.79382147 0.81259601 0.58900836 0.61136713 0.7822154  0.76907322
 0.72913466 0.61085509 0.42106161 0.49854924 0.46902202 0.51305684
 0.66052227 0.73357228 0.76224612 0.58934972 0.78033794 0.66922683
 0.71735791 0.70626387 0.6656426  0.63833419 0.52995392 0.74961598
 0.6275815  0.77368152 0.74739717 0.70472777 0.73032941 0.6344086
 0.58405871 0.7776071  0.56938044 0.62553337 0.66069295 0.58201058
 0.75917392 0.77982591 0.76531831 0.7166752  0.7776071  0.59310463
 0.65557262 0.73357228 0.60590544 0.61648746 0.776071   0.68748933
 0.58696023 0.58610684 0.57791432 0.6572794  0.61990101 0.88684076
 0.89281447 0.64516129 0.56545486 0.54548558 0.52790579 0.65762075
 0.72742789 0.65540195 0.70506912 0.54292541]
Feedback idx at HITL iteration 1: [35 58 10 54 52 44  7  4 30 31 48 50 63 42  5 41 47 21 36 28]
Human score at HITL iteration 1: [0.006252611738220664, 0.09562572122829235, 0.0019976969763088204, 0.0003059190130396286, 0.001093922891464353, 0.026633211028240445, 0.014716866909092217, 

Found local copy...


[0.77009729 0.62229049 0.40723673 0.67366445 0.77112135 0.60215054
 0.72537976 0.73374296 0.479092   0.51766513 0.46646185 0.53934118
 0.66803209 0.70899471 0.76719577 0.60727087 0.69977812 0.65676737
 0.60300393 0.66513057 0.74603175 0.58030381 0.52278546 0.79194402
 0.5666496  0.7646356  0.7593446  0.75405359 0.69158559 0.55128862
 0.60402799 0.57006315 0.62655743 0.57245264 0.55265404 0.53063663
 0.84707288 0.79023724 0.74159413 0.81259601 0.81669227 0.55265404
 0.65506059 0.59959037 0.53251408 0.69397508 0.84502475 0.7334016
 0.64021164 0.78767708 0.6359447  0.71940604 0.67520055 0.7776071
 0.84912101 0.57586619 0.6062468  0.5369517  0.51459293 0.65847414
 0.64789213 0.79006656 0.98003072 0.78153268]
Feedback idx at HITL iteration 2: [29 25  2 39 23 37 24  6 18 11 19  9 51 14  1 57 40 45 27 17]
Human score at HITL iteration 2: [0.0070444686987852475, 0.00011636932131217222, 0.0019002639556290404, 0.00032858436307791613, 0.0005927285195964616, 0.006062564241760079, 0.022833808903426

Found local copy...


[0.66222905 0.49871992 0.39614269 0.69226831 0.76873187 0.54326677
 0.66376515 0.78426353 0.48608978 0.45963475 0.47311828 0.54104796
 0.68066223 0.77043864 0.72452637 0.64789213 0.71462707 0.69192695
 0.55299539 0.59839563 0.77726575 0.54958184 0.50691244 0.78306878
 0.54019457 0.74278887 0.77231609 0.70250896 0.6489162  0.52227343
 0.6260454  0.50605905 0.67485919 0.58730159 0.60112647 0.52756443
 0.85816692 0.73766854 0.77811913 0.83051715 0.81993514 0.56682028
 0.62348524 0.60215054 0.51766513 0.75883257 0.88052569 0.78392217
 0.71309097 0.87267452 0.69482847 0.81293736 0.73271889 0.74449565
 0.79313876 0.60590544 0.62314388 0.59054446 0.53149002 0.65062297
 0.59515276 0.68134494 0.99658645 0.85065711]
Feedback idx at HITL iteration 3: [34 60 13 53 56 59 22 15 32  3 33 38 43 46  8 49 12 55 16 20]
Human score at HITL iteration 3: [0.029065209230810872, 0.00857194631411068, 0.010267681077359495, 0.0005152664286259124, 0.1530126038543241, 0.003212821839023997, 0.00450576750945881, 0.0

ValueError: Cannot take a larger sample than population when 'replace=False'